In [6]:
import pandas as pd
import numpy as np
from progressbar import progressbar as pb

df = pd.read_csv('all_data_fixed3.csv')

In [7]:
print(len(df))
df.head()

5330


,0,1,2,3,4,5
0,Совещательный орган при императоре в начале XX в,Сенат,Государственный совет,Консилия министров,Верховный тайный совет,2
1,Министр внутренних дел с 1904 г либерал автор ...,Лорис Меликов,Святополк Мирский,Милюков,Витте,2
2,Какое из названных событий произошло 27 января...,подрыв флагманского корабля Петропавловск,высадка японских войск на Ляодунском полуострове,героический бой Варяга и Корейца,заключение Портсмутского мирного договора,3
3,Что из названного относится к результатам перв...,появление многопартийности,формирование конституционной монархии,ликвидация сословного строя,введение рабочего контроля за производством,1
4,Для выхода из экономического кризиса вызванног...,ввело 8 часовой рабочий день в промышленности,ликвидировало помещичье землевладение,отменило выкупные платежи,ввело продовольственную разверстку,4


In [10]:
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer, SnowballStemmer
import re, string, unicodedata
import nltk
import inflect

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words


def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = SnowballStemmer('russian')
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def normalize(words):
    words = remove_punctuation(words)
    return words


def stem_and_lemmatize(words):
    stems = stem_words(words)
    lemmas = lemmatize_verbs(words)
    return stems, lemmas


def nlp_process(text):
    words = nltk.word_tokenize(text)
    stems, lemmas = stem_and_lemmatize(words)
    words = normalize(lemmas)
    return ' '.join(words)

In [11]:
embs = np.zeros((df.shape[0], df.shape[1]-1, 300))
embs.shape

(5330, 5, 300)

In [14]:
import gensim
import pymorphy2
import zipfile

model_file = 'rusvectores.zip'
with zipfile.ZipFile(model_file, 'r') as archive:
    stream = archive.open('model.bin')
    model = gensim.models.KeyedVectors.load_word2vec_format(stream, binary=True)

paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress


In [15]:
from ufal.udpipe import Model, Pipeline
import os
import re
import sys

def tag_ud(text='Текст нужно передать функции в виде строки!', modelfile="../russian-ud-2.0-170801.udpipe"):
    udpipe_model_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
    udpipe_filename = udpipe_model_url.split('/')[-1]

    if not os.path.isfile(modelfile):
        print('UDPipe model not found. Downloading...', file=sys.stderr)
        wget.download(udpipe_model_url)

    print('\nLoading the model...', file=sys.stderr)
    model = Model.load(modelfile)
    process_pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')

    print('Processing input...', file=sys.stderr)
    for line in text:
        # line = unify_sym(line.strip()) # здесь могла бы быть ваша функция очистки текста
        output = process(process_pipeline, text=line)
        print(' '.join(output))

In [16]:
import requests
import re

url = 'https://raw.githubusercontent.com/akutuzov/universal-pos-tags/4653e8a9154e93fe2f417c7fdb7a357b7d6ce333/ru-rnc.map'

mapping = {}
r = requests.get(url, stream=True)
for pair in r.text.split('\n'):
    pair = re.sub('\s+', ' ', pair, flags=re.U).split(' ')
    if len(pair) > 1:
        mapping[pair[0]] = pair[1]

print(mapping)

{'A': 'ADJ', 'ADV': 'ADV', 'ADVPRO': 'ADV', 'ANUM': 'ADJ', 'APRO': 'DET', 'COM': 'ADJ', 'CONJ': 'SCONJ', 'INTJ': 'INTJ', 'NONLEX': 'X', 'NUM': 'NUM', 'PART': 'PART', 'PR': 'ADP', 'S': 'NOUN', 'SPRO': 'PRON', 'UNKN': 'X', 'V': 'VERB'}


In [19]:
from pymystem3 import Mystem

def tag_mystem(text='Текст нужно передать функции в виде строки!'):  
    m = Mystem()
    processed = m.analyze(text)
    tagged = []
    for w in processed:
        try:
            lemma = w["analysis"][0]["lex"].lower().strip()
            pos = w["analysis"][0]["gr"].split(',')[0]
            pos = pos.split('=')[0].strip()
            if pos in mapping:
                tagged.append(lemma + '_' + mapping[pos])
            else:
                tagged.append(lemma + '_X')
        except:
            continue
    return tagged

In [22]:
a = list(df.iloc[0,:])
a

['Совещательный орган при императоре в начале XX в',
 'Сенат',
 'Государственный совет',
 'Консилия министров',
 'Верховный тайный совет',
 2]

In [23]:
lst_df = df.values.tolist()
lst_df

[['Совещательный орган при императоре в начале XX в',
  'Сенат',
  'Государственный совет',
  'Консилия министров',
  'Верховный тайный совет',
  2],
 ['Министр внутренних дел с 1904 г либерал автор проекта О мерах к усовершенствованию государственного порядка',
  'Лорис Меликов',
  'Святополк Мирский',
  'Милюков',
  'Витте',
  2],
 ['Какое из названных событий произошло 27 января 1904 г',
  'подрыв флагманского корабля Петропавловск',
  'высадка японских войск на Ляодунском полуострове',
  'героический бой Варяга и Корейца',
  'заключение Портсмутского мирного договора',
  3],
 ['Что из названного относится к результатам первой российской революции',
  'появление многопартийности',
  'формирование конституционной монархии',
  'ликвидация сословного строя',
  'введение рабочего контроля за производством',
  1],
 ['Для выхода из экономического кризиса вызванного Первой мировой войной российское правительство в конце 1916 г',
  'ввело 8 часовой рабочий день в промышленности',
  'ликвиди

In [24]:
def get_vec(lst):
    res = np.zeros((5,300))
    for i in range(len(lst)-1):
        cur_vec = np.zeros(300)
        cur_tag = tag_mystem(lst[i])
        for c in cur_tag:
            try:
                cur_vec += model.get_vector(c)
            except:
                continue
        res[i,:] = cur_vec
    return res

In [25]:
from multiprocessing import Pool

p = Pool(6)

answ = p.map(get_vec, lst_df)

In [26]:
answ_np = np.array(answ)

import pickle

with open('rusvec_embed3.pickle', 'wb') as f:
    pickle.dump(answ_np, f)